## Load the Dataset

In [1]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='/home/j/Documents/MLotsawa/transliteration/data/pairs.csv', column_names=['bo', 'phon'])

In [2]:
dataset = dataset['train'].train_test_split(.1)

## Load the Checkpoint Tokenizer, Model, and Data Collator

In [3]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

2024-09-08 20:55:41.756715: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-08 20:55:41.857676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-08 20:55:41.900066: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-08 20:55:41.911502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-08 20:55:41.985407: I tensorflow/core/platform/cpu_feature_guar

## Add Tibetan to Tokenizer

In [4]:
# Tibetan characters to add
tibetan_chars = [
    # Consonants
    "ཀ", "ཁ", "ག", "ང", "ཅ", "ཆ", "ཇ", "ཉ", "ཏ", "ཐ", "ད", "ན", "པ", "པ", "ཕ", "བ", "མ",
    "ཙ", "ཚ", "ཛ", "ཝ", "ཞ", "ཟ", "འ", "ཡ", "ར", "ལ", "ཤ", "ཥ", "ས", "ཧ", "ཨ",

    # Subjoined Consonants
    "ྐ", "ྑ", "ྒ", "ྒྷ", "ྔ", "ྕ", "ྖ", "ྗ", "྘", "ྙ", "ྚ", "ྛ", "ྜ", "ྜྷ", "ྞ", "ྟ",
    "ྠ", "ྡ", "ྡྷ", "ྣ", "ྤ", "ྥ", "ྦ", "ྦྷ", "ྨ", "ྩ", "ྪ", "ྫ", "ྫྷ", "ྭ", "ྮ", "ྯ",
    "ྰ", "ྱ", "ྲ", "ླ", "ྴ", "ྵ", "ྶ", "ྷ", "ྸ", "ྐྵ", "ྺ", "ྻ", "ྼ", "྽", "྾", "྿",

    # Vowels
    "ི", "ཱི", "ུ", "ཱུ", "ྲྀ", "ཷ", "ླྀ", "ཹ", "ེ", "ཻ", "ོ", "ཽ", "ཾ", "ཿ",

    # Other Marks and Symbols
    "འ", "ཡ", "ར", "ལ", "ཤ", "ཥ", "ས", "ཧ", "ཨ",

    # Additional Tibetan Characters
    "ཀྵ", "ཁྵ", "གྵ", "ངྵ", "ཅྵ", "ཆྵ", "ཇྵ", "ཉྵ", "ཏྵ", "ཐྵ", "དྵ", "ནྵ", "པྵ", 
    "པྵ", "ཕྵ", "བྵ", "མྵ", "ཙྵ", "ཚྵ", "ཛྵ", "ཝྵ", "ཞྵ", "ཟྵ", "འྵ", "ཡྵ", "རྵ", 
    "ལྵ", "ཤྵ", "ཥྵ", "སྵ", "ཧྵ", "ཨྵ", "པྪ", "པྫ", "པྫྷ", "པྭ", "པྮ", "པྯ", "པྰ", 
    "པྱ", "པྲ", "པླ", "པྴ", "པྵ", "པྶ", "པྷ", "པྸ", "པྐྵ", "པྺ", "པྻ", "པྼ", "པ྽", 
    "པ྾", "པ྿"
]


#'ཀཁགངཅཆཇཉཏཐདནཔཕབམཙཚཛཝཞཟའཡརལཤཥསཧཨ'

# Add the Tibetan characters to the tokenizer's vocabulary
new_tokens = [char for char in tibetan_chars if char not in tokenizer.get_vocab()]

# Add new tokens to the tokenizer
tokenizer.add_tokens(new_tokens)

# Resize model embeddings to accommodate the new vocabulary size
model.resize_token_embeddings(len(tokenizer))

Embedding(32245, 512)

## Preprocess Data

In [5]:
source_lang = 'bo'
target_lang = 'phon'

def preprocess_function(examples):

    inputs = [example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    
    model_inputs = tokenizer(inputs, text_target=targets, max_length=256, truncation=True, padding="max_length")

    return model_inputs


In [6]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/88737 [00:00<?, ? examples/s]

Map:   0%|          | 0/9860 [00:00<?, ? examples/s]

## Train the Model

In [7]:
from accelerate import Accelerator
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

accelerator = Accelerator()
model, optimizer = accelerator.prepare(model, optimizer)

training_args = Seq2SeqTrainingArguments(
    output_dir=f"../../models/tib-tokenized",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=5
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


  0%|          | 0/55465 [00:00<?, ?it/s]

{'loss': 0.4358, 'grad_norm': 0.4028961658477783, 'learning_rate': 0.0002972955918146579, 'epoch': 0.05}
{'loss': 0.2264, 'grad_norm': 0.1836295872926712, 'learning_rate': 0.00029459118362931577, 'epoch': 0.09}
{'loss': 0.2168, 'grad_norm': 0.21932683885097504, 'learning_rate': 0.0002918867754439736, 'epoch': 0.14}
{'loss': 0.1795, 'grad_norm': 0.3370167315006256, 'learning_rate': 0.00028918236725863157, 'epoch': 0.18}
{'loss': 0.1289, 'grad_norm': 0.6756218671798706, 'learning_rate': 0.0002864779590732894, 'epoch': 0.23}
{'loss': 0.0865, 'grad_norm': 0.4279094636440277, 'learning_rate': 0.00028377355088794737, 'epoch': 0.27}
{'loss': 0.0738, 'grad_norm': 0.26090720295906067, 'learning_rate': 0.0002810691427026052, 'epoch': 0.32}
{'loss': 0.0627, 'grad_norm': 0.3491724133491516, 'learning_rate': 0.0002783647345172631, 'epoch': 0.36}
{'loss': 0.0565, 'grad_norm': 0.2388933300971985, 'learning_rate': 0.000275660326331921, 'epoch': 0.41}
{'loss': 0.0462, 'grad_norm': 0.22108301520347595, 

  0%|          | 0/1233 [00:00<?, ?it/s]

{'eval_loss': 0.01764485239982605, 'eval_runtime': 115.2813, 'eval_samples_per_second': 85.53, 'eval_steps_per_second': 10.696, 'epoch': 1.0}
{'loss': 0.0244, 'grad_norm': 0.07438911497592926, 'learning_rate': 0.0002377986117371315, 'epoch': 1.04}
{'loss': 0.026, 'grad_norm': 0.10485607385635376, 'learning_rate': 0.0002350942035517894, 'epoch': 1.08}
{'loss': 0.0225, 'grad_norm': 0.14731422066688538, 'learning_rate': 0.0002323897953664473, 'epoch': 1.13}
{'loss': 0.0214, 'grad_norm': 0.29146984219551086, 'learning_rate': 0.00022968538718110516, 'epoch': 1.17}
{'loss': 0.0221, 'grad_norm': 0.2635507881641388, 'learning_rate': 0.0002269809789957631, 'epoch': 1.22}
{'loss': 0.0212, 'grad_norm': 0.27857252955436707, 'learning_rate': 0.00022427657081042096, 'epoch': 1.26}
{'loss': 0.0214, 'grad_norm': 0.08655886352062225, 'learning_rate': 0.00022157216262507886, 'epoch': 1.31}
{'loss': 0.0215, 'grad_norm': 0.0826745331287384, 'learning_rate': 0.00021886775443973676, 'epoch': 1.35}
{'loss': 

...

  0%|          | 0/1233 [00:00<?, ?it/s]

{'eval_loss': 0.008867141790688038, 'eval_runtime': 115.2943, 'eval_samples_per_second': 85.52, 'eval_steps_per_second': 10.694, 'epoch': 5.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'train_runtime': 18058.2619, 'train_samples_per_second': 24.57, 'train_steps_per_second': 3.071, 'train_loss': 0.028914398460556687, 'epoch': 5.0}


TrainOutput(global_step=55465, training_loss=0.028914398460556687, metrics={'train_runtime': 18058.2619, 'train_samples_per_second': 24.57, 'train_steps_per_second': 3.071, 'total_flos': 3.002456359305216e+16, 'train_loss': 0.028914398460556687, 'epoch': 5.0})